In [ ]:
!pip install -q transformers accelerate datasets peft bitsandbytes trl

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model


# Load and Prepare Dataset

dataset = load_dataset("json", data_files="/content/risk_cause_data_500.jsonl")["train"]

def format_example(example):
    return {"text": f"{example['prompt']}\n\nAssistant: {example['response']}"}

dataset = dataset.map(format_example)


# Load Tokenizer and Model

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


# Tokenize Dataset

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)


# QLoRA Config

lora_config = LoraConfig(
    r=16, # Rank
    lora_alpha=32, # Alpha
    target_modules=["query_key_value"], # Modules to apply LoRA to
    lora_dropout=0.05, # Dropout
    bias="none", # Bias
    task_type="CAUSAL_LM", # Task type
)

model = get_peft_model(model, lora_config)


# Fine-tuning Config

training_args = TrainingArguments(
    output_dir="./falcon-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    # fp16=True, # Disable fp16
    logging_steps=10,
    gradient_accumulation_steps=4, # Accumulate gradients
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# Final Trainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("falcon-finetuned")